<a href="https://colab.research.google.com/github/ClementBM/Experiment_GBDT_EarthQuake-LANL_Prediction/blob/master/LANL_Earthquake_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import some packages
Common packages and CatBoostRegressor

In [0]:
# math operations
import numpy as np
import math
# data preprocessing
import pandas as pd
# signal filtering
from scipy import signal as sg
import scipy
from scipy.signal import hilbert, chirp
from scipy.signal import find_peaks
from scipy import interpolate
from scipy import fftpack
# machine learning
from catboost import CatBoostRegressor, Pool
# serialization
import pickle
# io
from os import listdir
from os.path import isfile, join
import os
# parallel loop
from joblib import Parallel, delayed
import time
# data visualization
import matplotlib.pyplot as plt

### Functions for serialize and deserialize data

In [0]:
def deserialize(filename):
    f = open(filename, "rb")
    d = pickle.load(f)
    f.close()
    return d

def serialize(obj,filename):
    f = open(filename, "wb")
    d = pickle.dump(obj,f)
    f.close()
    return d

### Look at the data

In [0]:
# Data file path
trainPath = "train.csv"
testPath = "testdata"

# Get a sample of training data
trainSet = pd.read_csv(trainPath, nrows=2400000, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

# Plot based on both features

fig, ax1 = plt.subplots(figsize=(12, 8))
plt.title("Acoustic data and time to failure: 1% sampled data")
plt.plot(train['acoustic_data'], color='r')
ax1.set_ylabel('acoustic data', color='r')
plt.legend(['acoustic data'], loc=(0.01, 0.95))
ax2 = ax1.twinx()
plt.plot(train['time_to_failure'], color='b')
ax2.set_ylabel('time to failure', color='b')
plt.legend(['time to failure'], loc=(0.01, 0.9))
plt.grid(True)

plt.show()

### Features Engineering with FFT and statistic metrics

In [0]:
def GenFeatures(X):
    strain = []
    strain.append(X.mean())
    strain.append(X.kurtosis())
    strain.append(X.skew())
    strain.append(np.quantile(X,0.05))
    strain.append(np.quantile(X,0.95))
    strain.append(np.abs(X).mean())
    return strain

def GenFeaturesFromFft(X):
    strain = []
    strain.append(X.min()) 
    strain.append(X.kurtosis())
    strain.append(X.skew())
    strain.append(np.quantile(X,0.05))
    return strain

def CalculateFeatures(acousticData, timeToFailure):
    x = GenerateFeatures(acousticData)
    y = timeToFailure.values[-1]
    return x, y

def FourierTransform(x):
    fft = scipy.fftpack.fft(x)
    psd = np.abs(fft)
    fftfreq = scipy.fftpack.fftfreq(len(psd), 1.0/4_000_000)
    i = fftfreq > 0
    result = psd[i]
    return result

def GenerateFeatures(acousticData):    
    xDetrend = sg.detrend(acousticData)
    fft = FourierTransform(xDetrend)
    
    x2 = GenFeaturesFromFft(pd.Series(fft))
    x = np.concatenate((x1, x2), axis=0)
    
    xSerie = pd.Series(x)
    return xSerie

### Calculate features

In [0]:
number_lines = sum(1 for line in open(trainPath))
print(number_lines)

train = pd.read_csv(
    trainPath,
    iterator=True,
    chunksize=150_000,
    dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

xy = Parallel(n_jobs=-1)(delayed(CalculateFeatures)(df['acoustic_data'], df['time_to_failure']) for df in train)

x1, y1= zip(*xy)

X_train = pd.DataFrame()
y_train = pd.Series()

for i in range(len(x1)):
    X_train = X_train.append(x1[i], ignore_index=True)
    y_train = y_train.append(pd.Series(y1[i]))

print(X_train)
print(y_train)

serialize(X_train, "x_train-bak.bin")
serialize(y_train, "y_train-bak.bin")

### Build model and train on data

In [0]:
train_pool = Pool(X_train, y_train)
model = CatBoostRegressor(iterations=10000, loss_function='MAE', boosting_type='Ordered')
model.fit(X_train, y_train, silent=True)

serialize(model, "model-bak.bin")



### Evaluate model

In [0]:
print(model.best_score_)
print(model.feature_importances_)
print(model.tree_count_)

### Predict on testset

In [0]:
dataFiles = [f for f in listdir(testPath) if isfile(join(testPath, f))]

################################################
def Predict(testPath, dataFile):
    x = pd.read_csv(os.path.join(testPath, dataFile))
    xFeature = GenerateFeatures(x['acoustic_data'])
    prediction = model.predict(xFeature.to_frame().transpose())
    cell = []
    cell.append(dataFile.replace(".csv",""))
    cell.append(prediction[0])
    return cell

predictionsP = Parallel(n_jobs=-1)(delayed(Predict)(testPath, dataFile) for dataFile in dataFiles)

predictions = pd.DataFrame(columns=['seg_id', 'time_to_failure'])

for i in range(len(predictionsP)):
    predictions.loc[i] = predictionsP[i]
################################################

print(predictions)
predictions.to_csv("predictions-bak.csv", index=False)